# KDD Cup 1999 Data(標準化、次元圧縮、SVM)

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
from sklearn import datasets,svm, grid_search, metrics
from numpy import logspace
import sklearn.preprocessing as sp
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
import sklearn
from sklearn.decomposition import PCA
import time
% matplotlib inline

In [11]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.



|ファイル名|ファイル内容|
|---|---|
|kddcup.data|フルデータ|
|kddcup.data_10_percent|フルデータの10%を抽出した学習用データ|
|corrected|正常・攻撃のラベル付けがなされた評価用データ|
|kddcup.testdata.unlabeled|正常・攻撃のラベル付けがなされていないデータ|
|kddcup.testdata.unlabeled_10_percent|正常・攻撃のラベル付けがなされていないデータの10%サブセット|
|kddcup.newtestdata_10_percent_unlabeled|正常・攻撃のラベル付けがなされていないデータの10%サブセット|

In [12]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [13]:
kdd_data_10percent = joblib.load('dump/kddcup.data/kdd_data_10percent.00.pkl')


In [14]:
kdd_data_10percent.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,1,24,9,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,1,24,9,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,1,24,9,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,1,24,9,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,1,24,9,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [15]:
train_features = kdd_data_10percent.drop(['label','service'],axis=1)

In [16]:
train_labels = kdd_data_10percent['label'].copy()

In [17]:
train_labels.head()

0    normal.
1    normal.
2    normal.
3    normal.
4    normal.
Name: label, dtype: object

In [18]:
pipeline = Pipeline([
        ('standard_scaler', StandardScaler()),
    ('pca', PCA()),
        ('svm', SVC())])

In [21]:
score = 'f1'
params = {
    "pca__n_components" : list(range(2, 6)),
    'svm__kernel' : ["linear", "poly", "rbf", "sigmoid"],
    'svm__C' : np.logspace(0, 2, 5),
    'svm__gamma' : np.logspace(-3, 0, 5)
}

# Grid Searchを行う
clf = GridSearchCV(pipeline, params# 最適化したいパラメータセット 
    ,cv=5, # 交差検定の回数
    scoring='%s_weighted' % score ) # モデルの評価関数の指定



In [22]:
clf.fit(train_features[:5000], train_labels[:5000])

/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('standard_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'pca__n_components': [2, 3, 4, 5], 'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'svm__C': array([   1.     ,    3.16228,   10.     ,   31.62278,  100.     ]), 'svm__gamma': array([ 0.001  ,  0.00562,  0.03162,  0.17783,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1_weighted',
       verbose=0)

In [23]:
clf.grid_scores_

[mean: 0.99917, std: 0.00143, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.001, 'svm__kernel': 'linear'},
 mean: 0.99870, std: 0.00169, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.001, 'svm__kernel': 'poly'},
 mean: 0.99880, std: 0.00174, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.001, 'svm__kernel': 'rbf'},
 mean: 0.99880, std: 0.00174, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.001, 'svm__kernel': 'sigmoid'},
 mean: 0.99917, std: 0.00143, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.005623413251903491, 'svm__kernel': 'linear'},
 mean: 0.99910, std: 0.00156, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.005623413251903491, 'svm__kernel': 'poly'},
 mean: 0.99880, std: 0.00174, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.005623413251903491, 'svm__kernel': 'rbf'},
 mean: 0.99880, std: 0.00174, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma

In [24]:
print('Best_estimator = {0}'.format(clf.best_estimator_.get_params()))

Best_estimator = {'steps': [('standard_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svm', SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005623413251903491,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))], 'standard_scaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'pca': PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False), 'svm': SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005623413251903491,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False), 'standard_scaler__copy': True, 'standard_scaler__with_mean': True, 

In [25]:
clf.best_estimator_

Pipeline(steps=[('standard_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svm', SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005623413251903491,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [26]:
 
pred = clf.predict(train_features[:5000])



In [27]:
print(classification_report(train_labels[:5000], pred))
print(confusion_matrix(train_labels[:5000], pred))

                  precision    recall  f1-score   support

buffer_overflow.       1.00      1.00      1.00         2
     loadmodule.       1.00      1.00      1.00         1
         normal.       1.00      1.00      1.00      4996
           perl.       1.00      1.00      1.00         1

     avg / total       1.00      1.00      1.00      5000

[[   2    0    0    0]
 [   0    1    0    0]
 [   0    0 4996    0]
 [   0    0    0    1]]
